# Импорты

In [1]:
import numpy as np
from scipy.stats import norm
import networkx as nx
from collections import defaultdict
from matplotlib import pyplot as plt

# Импорт всех данных из варианта
from base.task_option import *

# Импорт общих методов для расчета связей
from utils.network_utils import create_network_with_dummies, calculate_positions, analyze_graph

# Класс сетевого анализа

In [2]:
class NetworkPlanning:
    def __init__(self):
        self.works = {}
        self.graph = nx.DiGraph()
        
    def add_work(self, name, dependencies, t_pes, t_ver, t_opt, cost_reduction=None):
        """Добавление работы с её характеристиками"""
        self.works[name] = {
            'dependencies': dependencies,
            't_pes': t_pes,
            't_ver': t_ver,
            't_opt': t_opt,
            'cost_reduction': cost_reduction
        }
    
    def calculate_three_param_model(self):
        """Расчет для трехпараметрической модели"""
        results = {}
        for name, work in self.works.items():
            t_oj = (work['t_pes'] + 4 * work['t_ver'] + work['t_opt']) / 6
            variance = ((work['t_pes'] - work['t_opt']) / 6) ** 2
            
            results[name] = {
                't_oj': round(t_oj),
                'variance': round(variance, 2)
            }
        return results
    
    def calculate_two_param_model(self):
        """Расчет для двухпараметрической модели"""
        results = {}
        for name, work in self.works.items():
            t_oj_star = (3 * work['t_pes'] + 2 * work['t_opt']) / 5
            variance = ((work['t_pes'] - work['t_opt']) / 6) ** 2
            
            results[name] = {
                't_oj_star': round(t_oj_star),
                'variance': round(variance, 2)
            }
        return results
    
    def build_network(self, model_type='three_param'):
        """Построение сетевого графика"""
        self.graph.clear()
        
        if model_type == 'three_param':
            model_results = self.calculate_three_param_model()
            time_key = 't_oj'
        else:
            model_results = self.calculate_two_param_model()
            time_key = 't_oj_star'
        
        # Создаем узлы на основе зависимостей
        node_counter = 0
        event_nodes = {}
        
        # Начальное событие
        event_nodes['start'] = node_counter
        node_counter += 1
        
        # Создаем узлы для каждой работы
        for name, work in self.works.items():
            if work['dependencies'] == ['-']:
                start_event = 'start'
            else:
                # Для упрощения создаем узел после зависимостей
                start_event = f"after_{'_'.join(work['dependencies'])}"
            
            end_event = f"end_{name}"
            
            if start_event not in event_nodes:
                event_nodes[start_event] = node_counter
                node_counter += 1
            if end_event not in event_nodes:
                event_nodes[end_event] = node_counter
                node_counter += 1
            
            self.graph.add_edge(
                event_nodes[start_event], 
                event_nodes[end_event], 
                weight=model_results[name][time_key],
                duration=model_results[name][time_key],
                variance=model_results[name]['variance'],
                name=name
            )
        
        return self.graph
    
    def find_critical_path(self, model_type='three_param'):
        """Нахождение критического пути"""
        graph = self.build_network(model_type)
        
        try:
            # Находим самый длинный путь (критический путь)
            longest_path = nx.dag_longest_path(graph, weight='weight')
            critical_time = nx.dag_longest_path_length(graph, weight='weight')
            
            # Находим работы на критическом пути
            critical_works = []
            total_variance = 0
            
            for i in range(len(longest_path) - 1):
                edge_data = graph[longest_path[i]][longest_path[i+1]]
                critical_works.append(edge_data['name'])
                total_variance += edge_data['variance']
            
            sigma = np.sqrt(total_variance)
            
            return {
                'critical_time': critical_time,
                'critical_works': critical_works,
                'total_variance': total_variance,
                'sigma': sigma
            }
        except nx.NetworkXError:
            return None
    
    def calculate_probability(self, T_dir, model_type='three_param'):
        """Расчет вероятности выполнения проекта к заданному сроку"""
        critical_path = self.find_critical_path(model_type)
        if not critical_path:
            return None
        
        T_kr = critical_path['critical_time']
        sigma = critical_path['sigma']
        
        z = (T_dir - T_kr) / sigma
        laplas_func_aproctimation = 0.5
        probability = laplas_func_aproctimation + norm.cdf(z) - laplas_func_aproctimation
        
        return probability
    
    def guaranteed_interval(self, P=0.9973, model_type='three_param'):
        """Интервал гарантированного времени выполнения"""
        critical_path = self.find_critical_path(model_type)
        if not critical_path:
            return None
        
        T_kr = critical_path['critical_time']
        sigma = critical_path['sigma']
        
        if P == 0.9973:
            delta = 3 * sigma
        else:
            z = norm.ppf((1 + P) / 2)
            delta = z * sigma
        
        return (T_kr - delta, T_kr + delta)
    
    def max_duration_with_reliability(self, gamma=0.95, model_type='three_param'):
        """Максимальный срок выполнения с заданной надежностью"""
        critical_path = self.find_critical_path(model_type)
        if not critical_path:
            return None
        
        T_kr = critical_path['critical_time']
        sigma = critical_path['sigma']
        
        z_gamma = norm.ppf(gamma)
        T_max = T_kr + z_gamma * sigma
        
        return T_max
    
    def optimize_project_duration(self, target_duration, model_type='three_param'):
        """Оптимизация сроков проекта с минимальными затратами"""
        critical_path = self.find_critical_path(model_type)
        if not critical_path:
            return None
        
        current_duration = critical_path['critical_time']
        needed_reduction = current_duration - target_duration
        
        if needed_reduction <= 0:
            return {"total_cost": 0, "reductions": []}
        
    def analyze_cost_reduction(self, model_type='three_param'):
        """Анализ возможностей сокращения сроков и затрат"""
        critical_works = self.find_critical_path(model_type)['critical_works']
        
        reduction_options = []
        for work in critical_works:
            if self.works[work]['cost_reduction']:
                reduction_options.append({
                    'work': work,
                    'cost_per_day': self.works[work]['cost_reduction'],
                    'max_reduction': self.calculate_max_reduction(work)
                })
        
        # Сортировка по стоимости сокращения (от самой дешевой)
        return sorted(reduction_options, key=lambda x: x['cost_per_day'])
    
    def calculate_max_reduction(self, work_name):
        """Расчет максимального возможного сокращения работы"""
        work = self.works[work_name]
        # Максимальное сокращение = t_опт (минимальное время)
        current_time = (work['t_pes'] + 4 * work['t_ver'] + work['t_opt']) / 6
        max_reduction = current_time - work['t_opt']
        return max(0, max_reduction)
    
    def calculate_total_cost(self, duration, daily_cost, model_type='three_param'):
        """Расчет общей стоимости проекта"""
        base_duration = self.find_critical_path(model_type)['critical_time']
        
        if duration >= base_duration:
            return daily_cost * duration
        
        # Если срок сокращен, добавляем затраты на ускорение
        optimization_result = self.optimize_project_duration(duration, model_type)
        acceleration_cost = optimization_result['total_cost'] if optimization_result else 0
        
        return (daily_cost * duration) + acceleration_cost
    
    def calculate_early_times(self):
        # Топологическая сортировка узлов
        topo_order = list(nx.topological_sort(self.graph))

        # Инициализация словаря для хранения Tр
        early_times = {node: 0 for node in self.graph.nodes}

        # Проход по узлам в топологическом порядке
        for node in topo_order:
            # Все входящие ребра для текущего узла
            predecessors = list(self.graph.predecessors(node))

            # Если есть предшественники
            if predecessors:
                # Вычисляем максимальное значение (Tр предшественника + длительность работы)
                max_time = max(
                    early_times[p] + self.graph.edges[p, node]['duration']
                    for p in predecessors
                )
                early_times[node] = max_time

        return early_times
    
    def calculate_late_times_correct(self, early_times):
        # Инициализация словаря для хранения Tп
        late_times = {node: 0 for node in self.graph.nodes}

        # Установка Tп для завершающего узла равным его раннему сроку
        end_node = max(self.graph.nodes)
        late_times[end_node] = early_times[end_node]

        # Топологическая сортировка узлов в обратном порядке
        topo_order = list(reversed(list(nx.topological_sort(self.graph))))

        # Проход по узлам в обратном топологическом порядке
        for node in topo_order:
            # Пропускаем завершающий узел, он уже обработан
            if node == end_node:
                continue

            # Все исходящие ребра для текущего узла
            successors = list(self.graph.successors(node))

            # Если есть последующие узлы, вычисляем минимальное значение
            if successors:
                min_time = min(
                    late_times[s] - self.graph.edges[node, s]['duration']
                    for s in successors
                )
                late_times[node] = min_time
            # Если нет последующих узлов (кроме завершающего), присваиваем Tп завершающего узла
            else:
                late_times[node] = late_times[end_node]

        return late_times

# Основные расчеты

In [3]:
# Создаем экземпляр класса
np_solver = NetworkPlanning()

for work in works_data:
    np_solver.add_work(*work)

print("=== ТРЕХПАРАМЕТРИЧЕСКАЯ МОДЕЛЬ ===")

# Расчет ожидаемого времени и дисперсии
three_param = np_solver.calculate_three_param_model()
print("\nОжидаемое время и дисперсия (трехпараметрическая модель):")
for work, values in three_param.items():
    print(f"{work}: t_ож = {values['t_oj']}, D(x) = {values['variance']}")

# Критический путь
critical_3param = np_solver.find_critical_path('three_param')
if critical_3param:
    print(f"\nКритический путь: {critical_3param['critical_works']}")
    print(f"Ожидаемое время выполнения: {critical_3param['critical_time']} дней")
    print(f"Дисперсия критического пути: {critical_3param['total_variance']:.2f}")
    print(f"Среднеквадратическое отклонение: {critical_3param['sigma']:.2f}")
    
    # Вероятность выполнения к сроку
    prob_3param = np_solver.calculate_probability(T_dir, 'three_param')
    print(f"\nВероятность выполнения к сроку {T_dir} дней: {prob_3param:.2%}")
    
    # Интервал гарантированного выполнения
    interval_3param = np_solver.guaranteed_interval(0.9973, 'three_param')
    print(f"Интервал гарантированного выполнения (P=0.9973): {interval_3param[0]:.1f} - {interval_3param[1]:.1f} дней")
    
    # Максимальный срок с надежностью
    max_duration_3param = np_solver.max_duration_with_reliability(gamma, 'three_param')
    print(f"Максимальный срок с надежностью {gamma}: {max_duration_3param:.1f} дней")

print("\n=== ДВУХПАРАМЕТРИЧЕСКАЯ МОДЕЛЬ ===")

# Расчет для двухпараметрической модели
two_param = np_solver.calculate_two_param_model()
print("\nОжидаемое время и дисперсия (двухпараметрическая модель):")
for work, values in two_param.items():
    print(f"{work}: t*_ож = {values['t_oj_star']}, D(x) = {values['variance']}")

# Критический путь
critical_2param = np_solver.find_critical_path('two_param')
if critical_2param:
    print(f"\nКритический путь: {critical_2param['critical_works']}")
    print(f"Ожидаемое время выполнения: {critical_2param['critical_time']} дней")
    
    # Вероятность выполнения к сроку
    prob_2param = np_solver.calculate_probability(T_dir, 'two_param')
    print(f"\nВероятность выполнения к сроку {T_dir} дней: {prob_2param:.2%}")
    
    # Интервал гарантированного выполнения
    interval_2param = np_solver.guaranteed_interval(0.9973, 'two_param')
    print(f"Интервал гарантированного выполнения (P=0.9973): {interval_2param[0]:.1f} - {interval_2param[1]:.1f} дней")
    
    # Максимальный срок с надежностью
    max_duration_2param = np_solver.max_duration_with_reliability(gamma, 'two_param')
    print(f"Максимальный срок с надежностью {gamma}: {max_duration_2param:.1f} дней")

print("\n=== СРАВНЕНИЕ РЕЗУЛЬТАТОВ ===")
if critical_3param and critical_2param:
    print(f"Разница в ожидаемом времени: {critical_2param['critical_time'] - critical_3param['critical_time']} дней")
    print(f"Разница в вероятности выполнения: {prob_2param - prob_3param:.2%}")

=== ТРЕХПАРАМЕТРИЧЕСКАЯ МОДЕЛЬ ===

Ожидаемое время и дисперсия (трехпараметрическая модель):
b1: t_ож = 6, D(x) = 0.69
b2: t_ож = 4, D(x) = 0.44
b3: t_ож = 10, D(x) = 1.36
b4: t_ож = 2, D(x) = 0.25
b5: t_ож = 4, D(x) = 0.25
b6: t_ож = 8, D(x) = 1.0
b7: t_ож = 3, D(x) = 0.69
b8: t_ож = 4, D(x) = 1.36
b9: t_ож = 6, D(x) = 1.36
b10: t_ож = 5, D(x) = 0.44
b11: t_ож = 9, D(x) = 1.0

Критический путь: ['b3']
Ожидаемое время выполнения: 10 дней
Дисперсия критического пути: 1.36
Среднеквадратическое отклонение: 1.17

Вероятность выполнения к сроку 26 дней: 100.00%
Интервал гарантированного выполнения (P=0.9973): 6.5 - 13.5 дней
Максимальный срок с надежностью 0.95: 11.9 дней

=== ДВУХПАРАМЕТРИЧЕСКАЯ МОДЕЛЬ ===

Ожидаемое время и дисперсия (двухпараметрическая модель):
b1: t*_ож = 6, D(x) = 0.69
b2: t*_ож = 4, D(x) = 0.44
b3: t*_ож = 10, D(x) = 1.36
b4: t*_ож = 3, D(x) = 0.25
b5: t*_ож = 4, D(x) = 0.25
b6: t*_ож = 8, D(x) = 1.0
b7: t*_ож = 4, D(x) = 0.69
b8: t*_ож = 6, D(x) = 1.36
b9: t*_ож = 

# Блок экономических расчетов

In [4]:
# Расчет для трехпараметрической модели
if critical_3param:
    base_duration_3param = critical_3param['critical_time']
    base_cost_3param = base_duration_3param * S_k
    
    print(f"\nТрехпараметрическая модель:")
    print(f"Базовая стоимость проекта: {base_cost_3param} д.е. ({base_duration_3param} дней × {S_k} д.е./день)")
    
    # Анализ сокращения сроков
    if T_dir < base_duration_3param:
        needed_reduction = base_duration_3param - T_dir
        print(f"Требуется сокращение на {needed_reduction} дней для выполнения к сроку {T_dir} дней")
        
        # Здесь можно добавить вызов метода optimize_project_duration
        # для расчета минимальной стоимости сокращения
    else:
        excess_days = T_dir - base_duration_3param
        print(f"Проект уложится в срок с запасом {excess_days} дней")

# Расчет для двухпараметрической модели  
if critical_2param:
    base_duration_2param = critical_2param['critical_time']
    base_cost_2param = base_duration_2param * S_k
    
    print(f"\nДвухпараметрическая модель:")
    print(f"Базовая стоимость проекта: {base_cost_2param} д.е. ({base_duration_2param} дней × {S_k} д.е./день)")

# Анализ стоимости сокращения работ
print(f"\nАнализ стоимости сокращения работ:")
print("Работа | Стоимость сокращения на 1 день")
print("-" * 40)
for work in works_data:
    name, _, _, _, _, cost_red = work
    if cost_red:
        print(f"{name:6} | {cost_red:2} д.е.")

# Расчет экономической целесообразности сокращения
print(f"\nЭкономическая целесообразность сокращения:")
for work in works_data:
    name, _, _, _, _, cost_red = work
    if cost_red and cost_red < S_k:
        print(f"- {name}: выгодно сокращать (экономия {S_k - cost_red} д.е. в день)")
    elif cost_red:
        print(f"- {name}: невыгодно сокращать (доп. затраты {cost_red - S_k} д.е. в день)")


Трехпараметрическая модель:
Базовая стоимость проекта: 100 д.е. (10 дней × 10 д.е./день)
Проект уложится в срок с запасом 16 дней

Двухпараметрическая модель:
Базовая стоимость проекта: 100 д.е. (10 дней × 10 д.е./день)

Анализ стоимости сокращения работ:
Работа | Стоимость сокращения на 1 день
----------------------------------------
b1     |  4 д.е.
b2     |  8 д.е.
b3     |  5 д.е.
b4     |  6 д.е.
b5     |  7 д.е.
b6     |  4 д.е.
b7     | 10 д.е.
b8     |  9 д.е.
b9     |  5 д.е.
b10    |  8 д.е.
b11    |  7 д.е.

Экономическая целесообразность сокращения:
- b1: выгодно сокращать (экономия 6 д.е. в день)
- b2: выгодно сокращать (экономия 2 д.е. в день)
- b3: выгодно сокращать (экономия 5 д.е. в день)
- b4: выгодно сокращать (экономия 4 д.е. в день)
- b5: выгодно сокращать (экономия 3 д.е. в день)
- b6: выгодно сокращать (экономия 6 д.е. в день)
- b7: невыгодно сокращать (доп. затраты 0 д.е. в день)
- b8: выгодно сокращать (экономия 1 д.е. в день)
- b9: выгодно сокращать (экономия

# Создаем направленный граф и добавляем узлы

In [ ]:
# Получаем ранние и поздние сроки
early_times = np_solver.calculate_early_times()
late_times = np_solver.calculate_late_times_correct(early_times)

# Использование:
G, dummy_tasks, work_to_node = create_network_with_dummies(works_data)

print("Фиктивные работы:")
for dummy in dummy_tasks:
    print(f"  {dummy[0]} -> {dummy[1]}")

print("\nРеальные работы в графе:")
for u, v, data in G.edges(data=True):
    if data['real']:
        print(f"  {u} -> {v}: {data['task']} (длительность: {data['duration']:.2f})")

print("\nФиктивные работы в графе:")
for u, v, data in G.edges(data=True):
    if not data['real']:
        print(f"  {u} -> {v}: {data['task']}")

Граф корректен, циклов не обнаружено
Фиктивные работы:

Реальные работы в графе:
  start -> work_0: b1 (длительность: 5.83)
  start -> work_1: b2 (длительность: 4.00)
  work_0 -> work_2: b3 (длительность: 9.83)
  work_0 -> work_3: b4 (длительность: 2.17)
  work_1 -> work_5: b6 (длительность: 8.33)
  work_2 -> work_9: b10 (длительность: 5.00)
  work_3 -> work_4: b5 (длительность: 3.83)
  work_3 -> work_6: b7 (длительность: 3.17)
  work_3 -> work_9: b10 (длительность: 5.00)
  work_4 -> work_8: b9 (длительность: 6.17)
  work_4 -> work_10: b11 (длительность: 8.67)
  work_5 -> work_8: b9 (длительность: 6.17)
  work_5 -> work_10: b11 (длительность: 8.67)
  work_6 -> work_7: b8 (длительность: 4.50)
  work_7 -> work_8: b9 (длительность: 6.17)
  work_7 -> work_10: b11 (длительность: 8.67)
  work_8 -> work_9: b10 (длительность: 5.00)

Фиктивные работы в графе:
  work_9 -> end: end
  work_10 -> end: end


In [6]:
# Анализируем граф перед визуализацией
analyze_graph(G)

# Затем используйте calculate_positions
pos = calculate_positions(G, dummy_tasks)

Количество узлов: 13
Количество ребер: 19
Есть циклы: False


# Визуализация

In [7]:
plt.figure(figsize=(14, 8))

# Разделяем реальные и фиктивные работы
real_edges = [(u, v) for u, v, d in G.edges(data=True) if d['real']]
dummy_edges = [(u, v) for u, v, d in G.edges(data=True) if not d['real']]

# Выделяем критические работы
critical_real_edges = [edge for edge in real_edges if edge in critical_edges]

# Рисуем узлы
nx.draw_networkx_nodes(G, pos, node_size=2000, node_color='lightblue')
nx.draw_networkx_labels(G, pos, font_size=12, font_weight='bold')

# Рисуем реальные работы (синие сплошные)
nx.draw_networkx_edges(G, pos, edgelist=real_edges, edge_color='blue',
                      width=3, arrowstyle='->', arrowsize=40)

# Рисуем критические работы (зеленые сплошные, толще)
nx.draw_networkx_edges(G, pos, edgelist=critical_real_edges, edge_color='green',
                      width=3, arrowstyle='->', arrowsize=40)

# Рисуем фиктивные работы (красные пунктирные)
nx.draw_networkx_edges(G, pos, edgelist=dummy_edges, edge_color='red',
                      width=3, style='dashed', arrowstyle='->', arrowsize=40)

# Подписи для реальных работ
real_labels = {(u, v): f"{d['task']}\n({d['duration']})" for u, v, d in G.edges(data=True) if d['real']}
nx.draw_networkx_edge_labels(G, pos, edge_labels=real_labels, font_color='blue')

# Подписи для критических работ (зеленые)
critical_labels = {(u, v): f"{G.edges[u, v]['task']}\n({G.edges[u, v]['duration']})" for u, v in critical_real_edges}
nx.draw_networkx_edge_labels(G, pos, edge_labels=critical_labels, font_color='green')

# Подписи для фиктивных работ
dummy_labels = {(u, v): d['task'] for u, v, d in G.edges(data=True) if not d['real']}
nx.draw_networkx_edge_labels(G, pos, edge_labels=dummy_labels, font_color='red')

# Добавление меток для ранних и поздних сроков
early_labels = {node: f"Tр: {early_times[node]}" for node in G.nodes}
late_labels = {node: f"Tп: {late_times[node]}" for node in G.nodes}

# Позиции для меток
early_pos = {node: (x, y + 0.02) for node, (x, y) in pos.items()}
late_pos = {node: (x, y - 0.02) for node, (x, y) in pos.items()}

# Рисуем метки для ранних сроков
nx.draw_networkx_labels(G, early_pos, labels=early_labels, font_size=8)

# Рисуем метки для поздних сроков
nx.draw_networkx_labels(G, late_pos, labels=late_labels, font_size=8)
plt.title("Граф работ")
plt.axis('off')
plt.show()

NameError: name 'critical_edges' is not defined

<Figure size 1400x800 with 0 Axes>